#Fine-tuning T5 from the Huggingface Library Simple Transformers

This notebook trains T5 on positive OpSpam data, and tests on negative OpSpam data

In [ ]:
!pip install simpletransformers
import pandas as pd
from simpletransformers.t5 import T5Model
from pprint import pprint
import logging
# Making sure the environment is set up correctly for anyone running this notebook
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf
import datetime as datetime
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
import sklearn
from google.colab import drive
from tensorflow import keras
import re

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Load Data**

In [ ]:
opspam_neg=pd.read_pickle('/content/drive/My Drive/6862_FakeReviewDetection/data/raw/opspam_neg.pkl')
opspam_pos=pd.read_pickle('/content/drive/My Drive/6862_FakeReviewDetection/data/raw/opspam_pos.pkl')

opspam_pos_train=pd.read_pickle('/content/drive/My Drive/6862_FakeReviewDetection/data/raw/opspam_pos_train.pkl')
opspam_pos_test=pd.read_pickle('/content/drive/My Drive/6862_FakeReviewDetection/data/raw/opspam_pos_test.pkl')

opspam_neg_train=pd.read_pickle('/content/drive/My Drive/6862_FakeReviewDetection/data/raw/opspam_neg_train.pkl')
opspam_neg_test=pd.read_pickle('/content/drive/My Drive/6862_FakeReviewDetection/data/raw/opspam_neg_test.pkl')

**Train the model**

In [ ]:
# Model arguments. Be careful to set arguments that allow the use of custom metrics.  
# https://simpletransformers.ai/docs/t5-model/
# We use convention-based defaults for the training batch size, and the number of epochs
# Using a small # of epochs has a regularization effect (i.e. balances wanting to fit the training data well, and finding estimates
# for parameters that generalize well 

model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 320,
    "train_batch_size": 32, #convention is 32; small batch sizes are noisy and offer a regularizing effect; the # of observations in the training dataset must be divisible by the train_batch_size
    "num_train_epochs": 3,
    "save_eval_checkpoints": True, 
    "save_model_every_epoch": True,
    # "silent": True,
    "evaluate_generated_text": True,
    "evaluate_during_training": True, 
    "evaluate_during_training_verbose": True, 
    #"num_workers": 1
}


In [ ]:
# Custom metrics function

def metrics_fn(l, p):
    # Change into integer types 
    l_int = np.array(l).astype(int)
    p_int = np.array(p).astype(int)

    eval_accuracy=sklearn.metrics.accuracy_score(l, p)
    f1_score =sklearn.metrics.f1_score(l, p, labels=['0', '1'], pos_label = '1')
    auc = sklearn.metrics.roc_auc_score(l_int, p_int)
    precision = sklearn.metrics.precision_score(l, p, pos_label = '1')
    recall = sklearn.metrics.recall_score(l, p, labels=['0', '1'], pos_label = '1')
    cm = sklearn.metrics.confusion_matrix(l, p, labels=['0', '1'])
    return {'accuracy': eval_accuracy,
            'f1_score': f1_score,
            'precision': precision,
            'recall': recall,
            'auc': auc,
            'confusion matrix': cm}
    #return sum([1 if label == pred else 0 for label, pred in zip(labels, preds)])

In [ ]:

# Instantiate the model #Hard to get GPU on colab
model = T5Model("t5","t5-small", args=model_args, use_cuda = True)

train_data = opspam_pos_train
test_data = opspam_pos_test
new_data = opspam_neg
train_data.target_text = train_data.target_text.astype(str)
test_data.target_text = test_data.target_text.astype(str)
new_data.target_text =new_data.target_text.astype(str)

# Train the model
# Common errors returned:
##                         mmap: cannnot allocate memory. In this case, "Restart runtime", and "Run all".
model.train_model(train_data, output_dir = "/content/drive/My Drive/6862_FakeReviewDetection/bestmodel_pos", eval_data=test_data, metrics = metrics_fn)


  0%|          | 0/640 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3226: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3226: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


Using Adafactor for T5


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/20 [00:00<?, ?it/s]

...

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3226: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


Generating outputs:   0%|          | 0/20 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3226: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


Decoding outputs:   0%|          | 0/160 [00:00<?, ?it/s]

(60,
 {'eval_loss': [0.32491695806384085, 0.15536717921495438, 0.16352499797940254],
  'global_step': [20, 40, 60],
  'metrics': [{'accuracy': 0.50625,
    'auc': 0.50625,
    'confusion matrix': array([[80,  0],
           [79,  1]]),
    'f1_score': 0.02469135802469136,
    'precision': 1.0,
    'recall': 0.0125},
   {'accuracy': 0.825, 'auc': 0.825, 'confusion matrix': array([[68, 12],
           [16, 64]]), 'f1_score': 0.8205128205128205, 'precision': 0.8421052631578947, 'recall': 0.8},
   {'accuracy': 0.79375, 'auc': 0.79375, 'confusion matrix': array([[70, 10],
           [23, 57]]), 'f1_score': 0.7755102040816327, 'precision': 0.8507462686567164, 'recall': 0.7125}],
  'train_loss': [0.2804558575153351, 0.216734379529953, 0.27477824687957764]})

**Evaluate**

In [ ]:
# Load model
#model = T5Model("t5", "/content/drive/My Drive/6862_FakeReviewDetection/bestmodel/checkpoint-1209-epoch-3")

In [ ]:
# Print out model metrics on training data
'''
result_train = model.eval_model(train_data, metrics = metrics_fn)
out = pd.DataFrame([result_train['metrics']])
out.to_csv("/content/drive/My Drive/6862_FakeReviewDetection/bestmodel_pos/m_training.csv")
print(out)
'''

'\nresult_train = model.eval_model(train_data, metrics = metrics_fn)\nout = pd.DataFrame([result_train[\'metrics\']])\nout.to_csv("/content/drive/My Drive/6862_FakeReviewDetection/bestmodel_pos/m_training.csv")\nprint(out)\n'

In [ ]:
# Print out model metrics on testing data
result_test = model.eval_model(test_data, metrics = metrics_fn)
out=pd.DataFrame([result_test['metrics']])
out.to_csv("/content/drive/My Drive/6862_FakeReviewDetection/bestmodel_pos/m_testing.csv")
print(out)

  0%|          | 0/160 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3226: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


Running Evaluation:   0%|          | 0/20 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/20 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3226: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


Decoding outputs:   0%|          | 0/160 [00:00<?, ?it/s]

   accuracy  f1_score  precision  recall      auc      confusion matrix
0   0.79375   0.77551   0.850746  0.7125  0.79375  [[70, 10], [23, 57]]


In [ ]:
# Print out model metrics on testing data
result_new = model.eval_model(new_data, metrics = metrics_fn)
out=pd.DataFrame([result_new['metrics']])
out.to_csv("/content/drive/My Drive/6862_FakeReviewDetection/bestmodel_pos/m_new.csv")
print(out)

  0%|          | 0/800 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3226: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3226: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


Running Evaluation:   0%|          | 0/100 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3226: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


Decoding outputs:   0%|          | 0/800 [00:00<?, ?it/s]

   accuracy  f1_score  precision  recall      auc         confusion matrix
0   0.64625  0.488246   0.882353  0.3375  0.64625  [[382, 18], [265, 135]]
